In [2]:
import math
import folium

In [3]:
#my_position = (2.3709011,48.8639243) # A day in Paris with half my friends
my_position = (138.515727,36.5978874) # A trip to Japan showing only close friends

friend_positions = [
        (2.3646355,48.8657311),
        (2.313652,48.88436),
        (2.2999191,48.87092530000001),
        (2.3601723,48.8677637),
        (2.3697853,48.8638113),
        (2.3600006,48.8644889),
        (2.3764801,48.8731835),
        (2.358799,48.8911324),
        (2.3479843000000002,48.8640372),
        (2.3121071,48.883795500000005),
        (2.3150253,48.8836827),
        (2.3400879,48.8425765),
        (2.3182869,48.8512748),
        (2.3697853,48.8638113),
        (2.3721886,48.8638113),
        (2.3709011,48.8639243),
        (2.3709011,48.8639243),
        (2.3707938,48.8639172),
        (2.3709011,48.8639243),
        (2.3709011,48.8639243),
        (2.3708904,48.8638537),
        (2.5090027,48.8981296),
        (2.5502014,48.8701349),
        (3.6886597,48.1569251),
        (3.0761719,50.6424939),
        (3.1036377,50.6215881),
        (2.3236084,48.7525672),
        (0.1867676,49.5323392),
        (0.12084960000000002,49.5287739),
        (-73.828125,41.5085773),
        (138.515625,36.5978891)]

In [4]:
friends_proportion = .3 #Adjust the proportion of friends you want to show
scan_scales = [
    100.0, 500.0, 1000.0, 1500.0,
    2000.0, 2500.0, 3000.0, 3500.0,
    4000.0, 4500.0, 5000.0, 6000.0,
    7000.0, 8000.0, 9000.0, 10000.0,
    12000.0, 14000.0, 16000.0, 18000.0,
    20000.0, 25000.0, 50000.0, 100000.0,
    500000.0, #What should be the max scale ? 500km ?
]

In [5]:
nautic_constant = (1852 * 60) * (1852 * 60)
max_scale = len(scan_scales) - 1
threshold = float(len(friend_positions)) * friends_proportion

In [6]:
friend_distances = [] 
for i, friend_position in enumerate(friend_positions):
    lat_diff = my_position[0] - friend_position[0]
    lon_diff = my_position[1] - friend_position[1]
    friend_distances.append((i, lat_diff * lat_diff + lon_diff * lon_diff))
#print(friend_distances)

In [7]:
#Must sort for the algorithm to be linear afterward
sorted_friend_distances = sorted(friend_distances, key=lambda x: x[1])
#print(sorted_friend_distances)

In [8]:
in_circle = 0
scale = 0
last_scale = 0
for (i, distance) in sorted_friend_distances:
    while scale <= max_scale and distance > (scan_scales[scale] * scan_scales[scale] / nautic_constant) :
        scale += 1        
    if scale > max_scale:
        break    
    in_circle += 1
    last_scale = scale
    if (in_circle >= threshold):
        break

#debug
in_circle = sum([1 if distance <= (scan_scales[last_scale] * scan_scales[last_scale] / nautic_constant) else 0 for (i, distance) in sorted_friend_distances])
print(threshold, in_circle, scale, last_scale, scan_scales[last_scale])

9.299999999999999 1 25 0 100.0


In [9]:
# Conversion
limit=scan_scales[last_scale]
my_coords=my_position[::-1]
friends_coords_list=[row[::-1] for row in friend_positions]

In [24]:
map_osm = folium.Map(location=my_coords, zoom_start=12, tiles="OpenStreetMap")

zoom = folium.CircleMarker(my_coords, popup='My zoom level', radius=limit, color='#7abf57', fill_color='#7abf57')
map_osm.add_children(zoom)
me = folium.CircleMarker(my_coords, popup='Me',radius=20, color='#3186cc',fill_color='#3186cc')
map_osm.add_children(me)
for row in friends_coords_list:
    friend = folium.CircleMarker(row, popup='A friend',radius=20, color='#cc5031',fill_color='#d47259',fill_opacity=1)
    map_osm.add_children(friend)
    
map_osm